In [10]:
import argparse
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import datetime
import time
import logging
from joblib import dump, load
from datetime import timedelta
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import RegressorChain
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import xgboost as xgb
import lightgbm as lightgbm




# data preperation
cantonKeys = ['AG','AI','AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG','ZH']
data = pd.read_csv("completedata.csv")
numberOfInputWeeks = 3
outputCategories = [
                  'case_inz_entries_7dayAverage',
                  'hosp_inz_entries_7dayAverage',
                  'death_inz_entries_7dayAverage',
                  'testPositvity',
                  'testPositvity_7dayAverageBoth',    
                  'transit_stations_percent_change_from_baseline_7dayAverage',
                  'workplaces_percent_change_from_baseline_7dayAverage',    
                   ]
numberOfOutputs = len(outputCategories)
numberOfPreComputedOutputWeeks = 4
split = numberOfOutputs * numberOfPreComputedOutputWeeks + 2
train_features = data[data['category']=='train'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
train_labels = data[data['category']=='train'].iloc[:,-split:-2]
validation1_features = data[data['category']=='validation 1'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
validation1_labels = data[data['category']=='validation 1'].iloc[:,-split:-2]
validation2_features = data[data['category']=='validation 2'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
validation2_labels = data[data['category']=='validation 2'].iloc[:,-split:-2]

pip = Pipeline([('minmax_scaler', MinMaxScaler())])
X_train = pip.fit_transform(train_features[train_features.columns].values)
X_valid1 = pip.transform(validation1_features[train_features.columns].values)
X_valid2 = pip.transform(validation2_features[train_features.columns].values)



# estimator list generation

numberOfOutputsForMultiOutput = numberOfPreComputedOutputWeeks
epochs = 1000
batch_size = 32 #16

'''
def getNormalKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(input_)
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(hidden1) if isMultiOutput else keras.layers.Dense(1)(hidden1)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model


def getDropoutKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(input_)
    dropout1 = keras.layers.Dropout(rate=0.5)(hidden1)
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(dropout1) if isMultiOutput else keras.layers.Dense(1)(dropout1)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def getRegularizedKerasModel(name, learningrate, nrOfNeurons, alpha, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu", kernel_regularizer=keras.regularizers.l2(alpha))(input_)
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(hidden1) if isMultiOutput else keras.layers.Dense(1)(hidden1)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def getTwoWayKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(input_)
    concat = keras.layers.Concatenate()([input_,hidden1])
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def getComplexKerasModel(name, learningrate, nrOfNeurons1, nrOfNeurons2, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons1, activation="relu")(input_)
    hidden2 = keras.layers.Dense(nrOfNeurons2, activation="relu")(hidden1)
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(hidden1) if isMultiOutput else keras.layers.Dense(1)(hidden2)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def getTwoWayComplexDropoutKerasModel(name, learningrate, nrOfNeurons1, nrOfNeurons2, nrOfNeurons3, nrOfNeurons4, nrOfNeurons5, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons1, activation="relu")(input_)
    dropout1 = keras.layers.Dropout(rate=0.5)(hidden1)
    hidden2 = keras.layers.Dense(nrOfNeurons2, activation="relu")(hidden1)
    dropout1 = keras.layers.Dropout(rate=0.5)(hidden2)
    hidden3 = keras.layers.Dense(nrOfNeurons3, activation="relu")(hidden2)
    dropout1 = keras.layers.Dropout(rate=0.5)(hidden3)
    hidden4 = keras.layers.Dense(nrOfNeurons4, activation="relu")(hidden3)
    dropout1 = keras.layers.Dropout(rate=0.5)(hidden4)
    hidden5 = keras.layers.Dense(nrOfNeurons5, activation="relu")(hidden4)
    dropout1 = keras.layers.Dropout(rate=0.5)(hidden5)
    concat = keras.layers.Concatenate()([input_,dropout1])
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def getDeepDropoutKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    dropout0 = keras.layers.Dropout(rate=0.3)(input_)
    hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout0)
    dropout1 = keras.layers.Dropout(rate=0.3)(hidden1)
    hidden2 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout1)
    dropout2 = keras.layers.Dropout(rate=0.3)(hidden2)
    hidden3 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout2)
    dropout3 = keras.layers.Dropout(rate=0.3)(hidden3)
    hidden4 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout3)
    dropout4 = keras.layers.Dropout(rate=0.3)(hidden4)
    hidden5 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout4)
    dropout5 = keras.layers.Dropout(rate=0.3)(hidden5)
    hidden6 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout5)
    dropout6 = keras.layers.Dropout(rate=0.3)(hidden6)
    hidden7 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout6)
    dropout7 = keras.layers.Dropout(rate=0.3)(hidden7)
    hidden8 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout7)
    dropout8 = keras.layers.Dropout(rate=0.3)(hidden8)
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(dropout8) if isMultiOutput else keras.layers.Dense(1)(dropout8)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model
'''

def generic1(learningrate, hiddenLayers, dropout, l1, l2, isTwoWay, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    dropout0 = keras.layers.Dropout(rate=dropout)(input_)
    
    hidden0 = keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout0)
    dropout1 = keras.layers.Dropout(rate=dropout)(hidden0)
    
    lastHidden = dropout1
    concat = keras.layers.Concatenate()([input_,lastHidden]) if isTwoWay else lastHidden
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic2(learningrate, hiddenLayers, dropout, l1, l2, isTwoWay, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    dropout0 = keras.layers.Dropout(rate=dropout)(input_)
    
    hidden0 = keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout0)
    dropout1 = keras.layers.Dropout(rate=dropout)(hidden0)
    hidden1 = keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout1)
    dropout2 = keras.layers.Dropout(rate=dropout)(hidden1)
    
    lastHidden = dropout2
    concat = keras.layers.Concatenate()([input_,lastHidden]) if isTwoWay else lastHidden
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic3(learningrate, hiddenLayers, dropout, l1, l2, isTwoWay, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    dropout0 = keras.layers.Dropout(rate=dropout)(input_)
    
    hidden0 = keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout0)
    dropout1 = keras.layers.Dropout(rate=dropout)(hidden0)
    hidden1 = keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout1)
    dropout2 = keras.layers.Dropout(rate=dropout)(hidden1)
    hidden2 = keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout2)
    dropout3 = keras.layers.Dropout(rate=dropout)(hidden2)
    
    lastHidden = dropout3
    concat = keras.layers.Concatenate()([input_,lastHidden]) if isTwoWay else lastHidden
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic4(learningrate, hiddenLayers, dropout, l1, l2, isTwoWay, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    dropout0 = keras.layers.Dropout(rate=dropout)(input_)
    
    hidden0 = keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout0)
    dropout1 = keras.layers.Dropout(rate=dropout)(hidden0)
    hidden1 = keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout1)
    dropout2 = keras.layers.Dropout(rate=dropout)(hidden1)
    hidden2 = keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout2)
    dropout3 = keras.layers.Dropout(rate=dropout)(hidden2)
    hidden3 = keras.layers.Dense(hiddenLayers[3], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout3)
    dropout4 = keras.layers.Dropout(rate=dropout)(hidden3)
    
    lastHidden = dropout4
    concat = keras.layers.Concatenate()([input_,lastHidden]) if isTwoWay else lastHidden
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic5(learningrate, hiddenLayers, dropout, l1, l2, isTwoWay, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    dropout0 = keras.layers.Dropout(rate=dropout)(input_)
    
    hidden0 = keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout0)
    dropout1 = keras.layers.Dropout(rate=dropout)(hidden0)
    hidden1 = keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout1)
    dropout2 = keras.layers.Dropout(rate=dropout)(hidden1)
    hidden2 = keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout2)
    dropout3 = keras.layers.Dropout(rate=dropout)(hidden2)
    hidden3 = keras.layers.Dense(hiddenLayers[3], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout3)
    dropout4 = keras.layers.Dropout(rate=dropout)(hidden3)
    hidden4 = keras.layers.Dense(hiddenLayers[4], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout4)
    dropout5 = keras.layers.Dropout(rate=dropout)(hidden4)
    
    lastHidden = dropout5
    concat = keras.layers.Concatenate()([input_,lastHidden]) if isTwoWay else lastHidden
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic6(learningrate, hiddenLayers, dropout, l1, l2, isTwoWay, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    dropout0 = keras.layers.Dropout(rate=dropout)(input_)
    
    hidden0 = keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout0)
    dropout1 = keras.layers.Dropout(rate=dropout)(hidden0)
    hidden1 = keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout1)
    dropout2 = keras.layers.Dropout(rate=dropout)(hidden1)
    hidden2 = keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout2)
    dropout3 = keras.layers.Dropout(rate=dropout)(hidden2)
    hidden3 = keras.layers.Dense(hiddenLayers[3], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout3)
    dropout4 = keras.layers.Dropout(rate=dropout)(hidden3)
    hidden4 = keras.layers.Dense(hiddenLayers[4], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout4)
    dropout5 = keras.layers.Dropout(rate=dropout)(hidden4)
    hidden5 = keras.layers.Dense(hiddenLayers[5], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout5)
    dropout6 = keras.layers.Dropout(rate=dropout)(hidden5)
    
    lastHidden = dropout6
    concat = keras.layers.Concatenate()([input_,lastHidden]) if isTwoWay else lastHidden
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic7(learningrate, hiddenLayers, dropout, l1, l2, isTwoWay, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    dropout0 = keras.layers.Dropout(rate=dropout)(input_)
    
    hidden0 = keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout0)
    dropout1 = keras.layers.Dropout(rate=dropout)(hidden0)
    hidden1 = keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout1)
    dropout2 = keras.layers.Dropout(rate=dropout)(hidden1)
    hidden2 = keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout2)
    dropout3 = keras.layers.Dropout(rate=dropout)(hidden2)
    hidden3 = keras.layers.Dense(hiddenLayers[3], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout3)
    dropout4 = keras.layers.Dropout(rate=dropout)(hidden3)
    hidden4 = keras.layers.Dense(hiddenLayers[4], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout4)
    dropout5 = keras.layers.Dropout(rate=dropout)(hidden4)
    hidden5 = keras.layers.Dense(hiddenLayers[5], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout5)
    dropout6 = keras.layers.Dropout(rate=dropout)(hidden5)
    hidden6 = keras.layers.Dense(hiddenLayers[6], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout6)
    dropout7 = keras.layers.Dropout(rate=dropout)(hidden6)
    
    lastHidden = dropout7
    concat = keras.layers.Concatenate()([input_,lastHidden]) if isTwoWay else lastHidden
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic8(learningrate, hiddenLayers, dropout, l1, l2, isTwoWay, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    dropout0 = keras.layers.Dropout(rate=dropout)(input_)
    
    hidden0 = keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout0)
    dropout1 = keras.layers.Dropout(rate=dropout)(hidden0)
    hidden1 = keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout1)
    dropout2 = keras.layers.Dropout(rate=dropout)(hidden1)
    hidden2 = keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout2)
    dropout3 = keras.layers.Dropout(rate=dropout)(hidden2)
    hidden3 = keras.layers.Dense(hiddenLayers[3], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout3)
    dropout4 = keras.layers.Dropout(rate=dropout)(hidden3)
    hidden4 = keras.layers.Dense(hiddenLayers[4], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout4)
    dropout5 = keras.layers.Dropout(rate=dropout)(hidden4)
    hidden5 = keras.layers.Dense(hiddenLayers[5], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout5)
    dropout6 = keras.layers.Dropout(rate=dropout)(hidden5)
    hidden6 = keras.layers.Dense(hiddenLayers[6], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout6)
    dropout7 = keras.layers.Dropout(rate=dropout)(hidden6)
    hidden7 = keras.layers.Dense(hiddenLayers[7], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2))(dropout7)
    dropout8 = keras.layers.Dropout(rate=dropout)(hidden7)
    
    lastHidden = dropout8
    concat = keras.layers.Concatenate()([input_,lastHidden]) if isTwoWay else lastHidden
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model




def genericKerasModel(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isTwoWay, isMultioutput):
    if len(hiddenLayers) == 1:
        return generic1(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isTwoWay, isMultioutput)
    elif len(hiddenLayers) == 2:
        return generic2(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isTwoWay, isMultioutput)
    elif len(hiddenLayers) == 3:
        return generic3(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isTwoWay, isMultioutput)
    elif len(hiddenLayers) == 4:
        return generic4(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isTwoWay, isMultioutput)
    elif len(hiddenLayers) == 5:
        return generic5(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isTwoWay, isMultioutput)
    elif len(hiddenLayers) == 6:
        return generic6(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isTwoWay, isMultioutput)
    elif len(hiddenLayers) == 7:
        return generic7(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isTwoWay, isMultioutput)
    else:
        return generic8(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isTwoWay, isMultioutput)

estimators = []
estimatorNames = []

# keras multi
'''
for learningrate in [0.001,0.0001,0.00001]:
        estimators.append(getNormalKerasModel("normal_learningrate_"+str(learningrate), learningrate, 100, True))
        estimators.append(getNormalKerasModel("normal_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, True))
        estimators.append(getDropoutKerasModel("dropout_learningrate_"+str(learningrate), learningrate, 100, True))
        estimators.append(getDropoutKerasModel("dropout_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, True))
        estimators.append(getTwoWayKerasModel("twoway_learningrate_"+str(learningrate), learningrate, 100, True))
        estimators.append(getTwoWayKerasModel("twoway_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, True))
        estimators.append(getComplexKerasModel("complex_learningrate_"+str(learningrate), learningrate, 100, 10, True))
        estimators.append(getComplexKerasModel("complex_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200)+"_"+str(20), learningrate, 200, 20, True))
        estimators.append(getTwoWayComplexDropoutKerasModel("twowaycomplexdropout_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size), learningrate, 200, 150, 100, 50, 10, True))
        estimators.append(getTwoWayComplexDropoutKerasModel("twowaycomplexdropout_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(300)+"_"+str(150)+"_"+str(100)+"_"+str(50)+"_"+str(20), learningrate, 300, 150, 100, 50, 20, True))
        estimators.append(getDeepDropoutKerasModel("DeepDropout"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(100), learningrate, 100, True))
        for alpha in [0.1,0.01, 0.001, 0.0001]: 
            estimators.append(getRegularizedKerasModel("regularized_learningrate_"+str(learningrate)+"_alpha_"+str(alpha)+"_batchsize_"+str(batch_size), learningrate, 100, alpha, True))     
            estimators.append(getRegularizedKerasModel("regularized_learningrate_"+str(learningrate)+"_alpha_"+str(alpha)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, alpha, True))             
'''
for lr in [0.001,0.0001]:
    for dropoutV in [0,0.3,0.5]: # 0.4
        for l1reg in [0.0001, 0.001, 0.01,0.1, 1, 0, 10]:
            for l2reg in [0.0001, 0.001, 0.01,0.1, 1, 0, 10]: #
                for twoWay in [True, False]:
                    for hiddenL in [
                                    [100],
                                    [150],
                                    [300]
                                   ]:
                        for multi in [True]:
                            name = "lr="+str(lr)+"_"+"hl="+str(hiddenL)+"_"+"do="+str(dropoutV)+"_"+"l1="+str(l1reg)+"_"+"l2="+str(l2reg)+"_"+"tw="+str(twoWay)+"_"+"mo="+str(multi)
                            estimatorNames.append(name)
                            estimators.append(genericKerasModel(lr,hiddenL,dropoutV, l1reg, l2reg, twoWay, multi))

for lr in [0.001,0.0001]:
    for dropoutV in [0,0.3,0.5]: 
        for l1reg in [0.0001, 0.01]:  
            for l2reg in [10]: 
                for twoWay in [True, False]:
                    for hiddenL in [[50],
                                    [200],
                                    [200,100],
                                    [100, 50],
                                    [200, 100, 50],
                                    [200, 150, 100, 50],
                                    [200, 150, 100, 75, 50],
                                    [200, 150, 100, 90, 80, 50],
                                    [200, 150, 100, 90, 80, 70, 50],
                                    [400, 200, 100, 90, 80, 70, 60, 50]
                                   ]:
                        for multi in [True]:
                            name = "lr="+str(lr)+"_"+"hl="+str(hiddenL)+"_"+"do="+str(dropoutV)+"_"+"l1="+str(l1reg)+"_"+"l2="+str(l2reg)+"_"+"tw="+str(twoWay)+"_"+"mo="+str(multi)
                            estimatorNames.append(name)
                            estimators.append(genericKerasModel(lr,hiddenL,dropoutV, l1reg, l2reg, twoWay, multi))



numberOfKerasEstimator = len(estimators)
print("Keras estimators: " + str(numberOfKerasEstimator))

# sklearn multi
'''
for alpha in [0.00001, 0.0001,0.001,0.01,0.1,0,1,10, 100]: 
    estimators.append(linear_model.Lasso(alpha=alpha))
    estimators.append(linear_model.Ridge(alpha=alpha))

kernels = ['linear','poly','polynomial','rbf','laplacian','sigmoid','cosine']
for alpha in [0.01,0.1,10]:
    for k in kernels:
        estimators.append(KernelRidge(kernel=k, alpha=alpha))

for alpha in [0.00001, 0.0001,0.001,0.01,0.1,1,10]: 
    estimators.append(linear_model.MultiTaskLasso(alpha=alpha))


for alpha in [0.00001, 0.0001,0.001,0.01,0.1,1,10]:
    for layer in [
        (20),
        (100),
        (20,20),
        (100,10),
        (100,50,10),
        (100,100,10),
    ]:
        estimators.append(MLPRegressor(hidden_layer_sizes=layer, alpha=alpha, max_iter=1000))

for nrOfEstimators in [100,200,500]: 
    for maxdepth in [None, 2, 4]:
        for mss in [2,3,4]:
            for msl in [1,2,3]:
                estimators.append(RandomForestRegressor(n_estimators=nrOfEstimators, 
                                                        max_depth=maxdepth, 
                                                        min_samples_split=mss, 
                                                        min_samples_leaf=msl,
                                                        n_jobs=-1))


'''
# training
numberOfSklearnEstimator = len(estimators) - numberOfKerasEstimator
print("Sklearn estimators: " + str(numberOfSklearnEstimator))

parser = argparse.ArgumentParser()
parser.add_argument("estimatorId", help="computes estimator with given id",type=int)
args = parser.parse_args()
estimatorId = args.estimatorId
'''
estimatorId = 0


else:
    for category in outputCategories:
        y_train = train_labels[["output_"+category+"_"+str(numberOfOutputWeeks) for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)]].values
        reg = estimator.fit(X_train, y_train)

        predictions_valid1 = pd.DataFrame(reg.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks)+"_"+category for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)]) 
        predictions_valid2 = pd.DataFrame(reg.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks)+"_"+category for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)])

        valid1Output = validation1_labels[["output_"+category+"_"+str(numberOfOutputWeeks) for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)]]
        valid2Output = validation2_labels[["output_"+category+"_"+str(numberOfOutputWeeks) for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)]]

        for outputWeekNumber in range(0,numberOfPreComputedOutputWeeks):
            rmse1 = np.sqrt(mean_squared_error(predictions_valid1["pred_week_" + str(outputWeekNumber)+"_"+category], valid1Output["output_"+category+"_"+str(outputWeekNumber)]))
            rmse2 = np.sqrt(mean_squared_error(predictions_valid2["pred_week_" + str(outputWeekNumber)+"_"+category], valid2Output["output_"+category+"_"+str(outputWeekNumber)]))

            d = {'model':[reg],'model type': ["sklearn_multi_weeks_output"],'target':[category],'week':[outputWeekNumber], 'model rsme 1':[rmse1], 'model rsme 2':[rmse2]}
            df = pd.DataFrame(data=d)
            df.to_csv("results.csv",mode='a', header=False, index=False)
'''

estimator = estimators[estimatorId]

#if (str(type(estimator)) == "<class 'tensorflow.python.keras.engine.functional.Functional'>"):
for category in outputCategories:
    y_train = train_labels[["output_"+category+"_"+str(numberOfOutputWeeks) for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)]].values
    y_valid1 = validation1_labels[["output_"+category+"_"+str(numberOfOutputWeeks) for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)]].values 
    estimator.fit(X_train, 
                  y_train, 
                  batch_size=batch_size, 
                  epochs=epochs, 
                  verbose=0, 
                  validation_data=(X_valid1,y_valid1), 
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])
    predictions_valid1 = pd.DataFrame(estimator.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks)+"_"+category for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)]) 
    predictions_valid2 = pd.DataFrame(estimator.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks)+"_"+category for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)])

    valid1Output = validation1_labels[["output_"+category+"_"+str(numberOfOutputWeeks) for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)]]
    valid2Output = validation2_labels[["output_"+category+"_"+str(numberOfOutputWeeks) for numberOfOutputWeeks in range(0,numberOfPreComputedOutputWeeks)]]

    for outputWeekNumber in range(0,numberOfPreComputedOutputWeeks):
        rmse1 = np.sqrt(mean_squared_error(predictions_valid1["pred_week_" + str(outputWeekNumber)+"_"+category], valid1Output["output_"+category+"_"+str(outputWeekNumber)]))
        rmse2 = np.sqrt(mean_squared_error(predictions_valid2["pred_week_" + str(outputWeekNumber)+"_"+category], valid2Output["output_"+category+"_"+str(outputWeekNumber)]))

        d = {'model':estimatorNames[estimatorId],'model type': ["keras_multi_weeks_output"],'target':[category],'week':[outputWeekNumber], 'model rsme 1':[rmse1], 'model rsme 2':[rmse2]}
        df = pd.DataFrame(data=d)
        df.to_csv("results.csv",mode='a', header=False, index=False)


usage: ipykernel_launcher.py [-h] estimatorId
ipykernel_launcher.py: error: argument estimatorId: invalid int value: '/home/david/.local/share/jupyter/runtime/kernel-779b8e2c-19a0-40ec-9187-78c4e401aa6a.json'
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Keras estimators: 2004
Sklearn estimators: 0
Traceback (most recent call last):
  File "/usr/lib/python3.9/argparse.py", line 2476, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: '/home/david/.local/share/jupyter/runtime/kernel-779b8e2c-19a0-40ec-9187-78c4e401aa6a.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.9/argparse.py", line 1851, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/usr/lib/python3.9/argparse.py", line 2063, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "/usr/lib/python3.9/argparse.py", line 2019, in consume_positionals
    take_action(action, args)
  File "/usr/lib/python3.9/argparse.py", line 1912, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "/usr/lib/python3.9/argparse.py", line 2443, in _get_values
    value

TypeError: object of type 'NoneType' has no len()